##  Introduction 
This dataset shows the statistics of NBA player from the year 2018 to September 2021 in the per match format and I will be using this dataset to answer a few simple questions. :) 

## Questions 
1. What is the age distribution of NBA players in 2021? 
2. Age vs Points per game
3. Ratio of positions 
4. TOP 20 3P shooters 

## 1. Importing csv file and installing neccesary packages

In [ ]:
import pandas as pd # data processing
import numpy as np # linear algebra
import sqlite3
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
# We will sqlite3 library and create a connection
cnn = sqlite3.connect('jupyter_sql_tutorial.db') # This is the database name we are using

In [ ]:
dataset = pd.read_csv('../input/nba-stats-2018-2021/NBA Stats 2018-2021/NBA_stats.csv')
dataset.head()

## 2. Data Cleaning
Let's start by viewing some basic information of the metadata.

In [ ]:
dataset.info()

In [ ]:
dataset.shape

### Missing Values 

In [ ]:
dataset.isnull().sum()

As we can see from the above, there is no null values in this dataset. Awesome! 

## 3. Connecting to SQLite database using magic command

In [ ]:
%load_ext sql

In [ ]:
#Create a SQL database
%sql sqlite:///jupyter_sql_tutorial.db 

In [ ]:
pat = r'C:\\Users\\Chum\\Desktop\\nba\\NBA_stats'
database = pat + 'database.sqlite'
cnn = sqlite3.connect(database)

In [ ]:
dataset.to_sql('nbadata', cnn)

In [ ]:
%%sql
SELECT *
FROM nbadata
LIMIT 20

## 4. Data Exploration and Visualizations



### i) Age distribution of players in 2021

In [ ]:
selected = ['Player','Age','Team','Pos','Points','Year']

Players_select = dataset[selected].sort_values(by ='Age', ascending=0)
Players_select.head(20)

In [ ]:
Players_2021 = Players_select[Players_select['Year']== 2021]
Players_2021

I am only interested in the 2021 season. So let's filter the data for only year 2021. 

We can also observe that some players play for more than one team, for those players, we will only take the row where the team = TOT. ‘TOT’ is just the cumulative score from all of the teams that the player played for that year.

### Multiple Rows for the Same Player

Here, we'll replace these multiple rows with a single row containing the averages across all of the player's teams. 

In [ ]:
df1 = Players_select[Players_select['Year']== 2021].groupby(['Player']).mean().sort_values(by= 'Age', ascending = 0)
df1

Now, we can see that the total rows has been reduced from 705 to 540 because we have collated the rows of players that played for more than 1 team and averaged the points. (For example, from the J.J Redick does not have 3 rows anymore.)

In [ ]:
fig = px.histogram(df1, x="Age")
fig.show()

From the distribution chart, the youngest age in the 2021 league is 19 years old, and the oldest player is 40 years old. 

## ii) Age vs Points per game
In general, physical performance declines as age increases. Let's see if this is true.

In [ ]:
fig = px.scatter(df1, y="Points", x="Age",
           title="Correlation Between Age and Performance",
           labels=dict(heightWOShoesInches="Age", wingspanInches= "Wingspan in Inches" ))
fig.show()

There are less players scoring more than 20 points as age increases beyond 30, but there are 2 outliers above the rest. Let's check out which players they are. 

In [ ]:
df1[(df1.Age ==32) & (df1.Points==32)]


In [ ]:
df1[(df1.Age ==36) & (df1.Points==25)]

STEPHEN CURRY AND KING JAMES. Not surprising at all, goats forsure.

## iii) Position distribution 

In [ ]:
#the sql query must be highlighted in red in the same line, if it is in the next line, it will appear black and will error
query = ("SELECT Player, Pos FROM nbadata WHERE Year = 2021;")
df_positions = pd.read_sql(query, cnn)
df_positions.head(20)

In [ ]:
df_positions.drop_duplicates()
query1 = ("SELECT COUNT(Player), Pos FROM nbadata GROUP BY Pos HAVING COUNT(Player)>10 ORDER BY COUNT(Player) asc ;")  #to exclude other mixed positions
df_positions2 = pd.read_sql(query1, cnn)
df_positions2.head(20)

In [ ]:
df_positions2.plot.bar(x="Pos", y="COUNT(Player)", title="Player Positions", figsize=(5,4));

plt.plot()
plt.xlabel('Position')
plt.ylabel('Total Players')

## iv) TOP 20 High Volume 3P shooters
Since we have the 3P and 3PA data, we will have to calculate the 3P%. 

In [ ]:
query2 = ("SELECT Player, YEAR, AVG(cast([3-points Field Goal] as decimal)) AS T_FG, AVG(cast([3-points Field Goal Attempted] as decimal)) AS T_FGA, AVG(cast([3-points Field Goal] as decimal))/AVG(cast([3-points Field Goal Attempted] as decimal)) AS TPP FROM nbadata WHERE cast([3-points Field Goal] as decimal) != 0 AND cast([3-points Field Goal Attempted] as decimal) != 0 AND YEAR=2021 GROUP BY Player, Year ORDER by T_FGA desc")
df_3PP = pd.read_sql(query2, cnn)
df_3PP.head(25)

In [ ]:
#x axis of the chart below was appearing as index numbers, but I want it to display player's names
#reset index by Player
df_3PP.set_index("Player",drop=True,inplace=True)

In [ ]:
#check
df_3PP.index.values

In [ ]:
fig, ax = plt.subplots(figsize = (16, 8))

ax.plot('TPP', data = df_3PP[:20], marker = 'o', color = 'darkblue', linewidth=2.5)
ax.axhline(y = np.mean(df_3PP['TPP']), linestyle = 'dashed', label = "League Average 3P%", color = 'red')

ax.set_xlabel('Player', fontsize = 14, labelpad = 20)
ax.set_ylabel('3-Point Percentage', fontsize = 14, labelpad = 20)
ax.set_title("Highest Volume 3-Point Shooters", fontsize = 20, pad = 20)
ax.grid(visible=True, which='both', axis='y')
ax.xaxis.set_ticks(df_3PP.index[:20])
ax.set_xticklabels(df_3PP.index[:20], rotation = 30)

ax.legend(loc = 'upper left', bbox_to_anchor = (1.05, 1), frameon = True)

plt.axis();

## Conclusion 
**1. What is the age distribution of NBA players in 2021?**
The age of players ranges from 19 - 40. 

**2. Age vs Points per game**
As age increases beyond 30, the ability to score above 20 points decreases. However, there are 2 special players above 30 years of age that scores higher than most younger players. These players are Stephen Curry and Lebron James.

**4. Distribution of positions**
The two most played positions are Shooting Guard, Power Forward, followed by Center, Point Guard and Small Forward.

**5. TOP 20 3P shooters**
The data visualisation displays the top 20 highest volume shooters with a 3-point percentage higher that the league average except for Eric Gordon falling slight below the average line.